In [20]:
import pandas as pd
import string
from string import digits
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import seaborn as sns


In [21]:
exclude = set(string.punctuation) # Set of all special characters 
# string.punctuations-->  (!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~)
exclude = list(exclude)#.append('€')
exclude.append('€')
print(exclude)
exclude = set(exclude)
print(exclude)

['|', ']', '#', '@', ':', '>', '&', '=', '[', '(', '?', '^', '<', '`', '.', '-', '%', '+', "'", ')', '!', '"', '$', '~', '*', ',', '{', '/', '_', '\\', ';', '}', '€']
{'|', ']', '#', '@', '>', ':', '&', '=', '[', '(', '?', '^', '<', '`', '.', '-', '%', '+', "'", ')', '!', '€', '"', '$', '~', '*', ',', '{', '/', '_', '\\', ';', '}'}


# import data and data cleaning

In [22]:

lines= pd.read_table('mar.txt', names=['eng', 'mar', 'cc'])
# print(lines.shape)
# print(lines)

# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: x.replace("'", ''))
lines.mar=lines.mar.apply(lambda x: x.replace("'", ''))

# Remove all the special characters
exclude = set(string.punctuation) # Set of all special characters 
# string.punctuations-->  (!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~)
exclude = list(exclude)
exclude.append('€')
exclude = set(exclude)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits_table = str.maketrans('', '', digits)
lines.eng = lines.eng.apply(lambda x: x.translate(remove_digits_table))
lines.mar = lines.mar.apply(lambda x: x.replace("[२३०८१५७९४६]", ""))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: x.replace(" +", " "))
lines.mar=lines.mar.apply(lambda x: x.replace(" +", " "))

# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')
# print(lines.eng.head)
# print(lines.mar.head)

lines.drop(columns='cc',inplace=True)
lines

,eng,mar
0,go,START_ जा _END
1,run,START_ पळ _END
2,run,START_ धाव _END
3,run,START_ पळा _END
4,run,START_ धावा _END
...,...,...
41023,just saying you dont like fish because of the ...,START_ हड्डींमुळे मासे आवडत नाही असं म्हणणं हे...
41024,the japanese parliament today officially elect...,START_ आज जपानी संसदेने अधिकृतरित्या र्‍यौतारौ...
41025,tom tried to sell his old vcr instead of throw...,START_ टॉमने त्याचा जुना व्हीसीआर फेकून टाकण्य...
41026,you cant view flash content on an ipad however...,START_ आयपॅडवर फ्लॅश आशय बघता येत नाही पण तुम्...


In [23]:
print(lines.sample(15))

                                          eng                                                mar
24362               we speak english in class            START_ आम्ही वर्गात इंग्रजीत बोलतो _END
31669         are you going to tell me or not        START_ तुम्ही मला सांगणार आहात की नाही _END
17785                  i am downloading books               START_ मी पुस्तकं डाउनलोड करतेय _END
7778                        it has cooled off                              START_ थंड झालाय _END
31285          tom answered all our questions  START_ टॉमने आपल्या सर्व प्रश्नांची उत्तरं दिल...
24011                theyre expected tomorrow                 START_ ते उद्या अपेक्षित आहेत _END
8390                        tom sells flowers                         START_ टॉम फुलं विकतो _END
12168                     start your own list                  START_ स्वतःची यादी सुरू करा _END
13494                    he is angry with you              START_ ते तुझ्यावर रागवलेले आहेत _END
37145  some people eat sushi w

# visualizing data for word to vector conversion and training

In [24]:
# Vocabulary of English
all_eng_words=set()
for line in lines.eng:
    for word in line.split():
      all_eng_words.add(word)
        # if word not in all_eng_words:
        #     all_eng_words.add(word)
# print(len(all_eng_words))
# print(all_eng_words)

# Vocabulary of Marathi 
all_marathi_words=set()
for line in lines.mar:
    for word in line.split():
      all_marathi_words.add(word)
        # if word not in all_marathi_words:
        #     all_marathi_words.add(word)
# print(all_marathi_words)

# Max Length of source sequence(length of sentences)
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
# print(max_length_src)

# Max Length of target sequence(length of marathi sentences)
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
# print(max_length_tar)

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
# print(input_words)
# print(target_words)

# Calculate Vocab size(number of distinct words) for both source and target
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_decoder_tokens += 1 # For zero padding
# print(num_encoder_tokens)
# print(num_decoder_tokens)

# Create word to token dictionary for both source and target
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])
print(input_token_index)
print(target_token_index)

# Create token to word dictionary for both source and target
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
# print(reverse_input_char_index)
# print(reverse_target_char_index)

{'a': 0, 'abbreviation': 1, 'ability': 2, 'able': 3, 'about': 4, 'above': 5, 'abroad': 6, 'absent': 7, 'absolute': 8, 'absolutely': 9, 'accent': 10, 'accept': 11, 'acceptable': 12, 'accepted': 13, 'access': 14, 'accessible': 15, 'accident': 16, 'accidentally': 17, 'accomplices': 18, 'according': 19, 'accordion': 20, 'account': 21, 'accused': 22, 'accusing': 23, 'accustomed': 24, 'ace': 25, 'aces': 26, 'acetone': 27, 'ache': 28, 'aches': 29, 'achille': 30, 'achilles': 31, 'acid': 32, 'acknowledged': 33, 'acquired': 34, 'acres': 35, 'across': 36, 'act': 37, 'acted': 38, 'acting': 39, 'action': 40, 'activist': 41, 'actor': 42, 'actors': 43, 'actress': 44, 'acts': 45, 'actual': 46, 'actually': 47, 'add': 48, 'added': 49, 'addict': 50, 'addicted': 51, 'addition': 52, 'additional': 53, 'addon': 54, 'address': 55, 'administrative': 56, 'admire': 57, 'admit': 58, 'admitted': 59, 'adopt': 60, 'adopted': 61, 'ads': 62, 'adultery': 63, 'advance': 64, 'advantage': 65, 'adverb': 66, 'advertisements

In [25]:
len(lines.eng)

41028

# treain test split

In [26]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
21389,he looks like his father,START_ तो आपल्या वडिलांसारखा दिसतो _END
19747,i know how to do my job,START_ माझं काम कसं करायचं हे मला माहीत आहे _END
35213,i just wanted to teach you a lesson,START_ मला फक्त तुम्हाला धडा शिकवायचा होता _END
13915,i will give you this,START_ मी तुला हे देईन _END
34545,ill never forget what you told me,START_ तुम्ही मला जे सांगितलंत ते मी कधीच विसर...
27091,nobody knows where he lives,START_ तो कुठे राहतो हे कोणालाच माहीत नाही _END
580,i see them,START_ मला ते दिसतात _END
19632,i do not need money now,START_ मला सध्या पैश्याची गरज नाही आहे _END
3098,i followed you,START_ मी तुमचा पाठलाग केला _END
32776,i just know his name thats all,START_ मला फक्त त्याचं नाव माहीत आहे तितकंच _END


In [27]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((36925,), (4103,), (36925,), (4103,))

In [28]:
print(type(X_train))
print(X_train.head)
print(y_train.head)

<class 'pandas.core.series.Series'>
<bound method NDFrame.head of 36606                some parts of this city are very ugly
36556                mary was looking for you at that time
31136                        shes used to getting up early
11310                                  he knows everything
9939                                    she goes to school
                               ...                        
40803    english is a universal language and is used al...
24759                           did you go to church today
5770                                      i almost drowned
20603                              tom picked up the phone
33423                    all sorts of people live in tokyo
Name: eng, Length: 36925, dtype: object>
<bound method NDFrame.head of 36606      START_ या शहराचे काही भाग अतिशय कुरूप आहेत _END
36556         START_ मेरी त्यावेळी तुम्हाला शोधत होती _END
31136                 START_ तिला लवकर उठायची सवय आहे _END
11310            START_ त्याला सर्व ग

# Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [29]:
import pickle
with open('X_train.pkl', 'wb') as file:
  X_train.to_pickle('X_train.pkl')
with open('X_test.pkl', 'wb') as file:
  X_test.to_pickle('X_test.pkl')

In [30]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float64')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float64')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float64')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# def generate_batch(X = X_train, y = y_train, batch_size = 128):
#     ''' Generate a batch of data '''
#     for j in range(0, 127, batch_size):
#         encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
#         decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
#         decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
#         for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
#             for t, word in enumerate(input_text.split()):
#                 encoder_input_data[i, t] = input_token_index[word] # encoder input seq
#             for t, word in enumerate(target_text.split()):
#                 if t<len(target_text.split())-1:
#                     decoder_input_data[i, t] = target_token_index[word] # decoder input seq
#                 if t>0:
#                     # decoder target sequence (one hot encoded)
#                     # does not include the START_ token
#                     # Offset by one timestep
#                     decoder_target_data[i, t - 1, target_token_index[word]] = 1.
#         yield([encoder_input_data, decoder_input_data], decoder_target_data)

# for [x,y],z in generate_batch():
#   print([x,y])
#   print(z)
  

# encoder decoder model architecture


In [31]:
latent_dim = 50

In [32]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Use a softmax to generate a probability distribution over the target vocabulary for each time step
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     284450      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     695250      input_4[0][0]                    
____________________________________________________________________________________________

In [33]:
# from IPython.display import Image
# Image(retina=True, filename='train_model.png')

In [34]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [38]:
model.fit(generate_batch(),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50
288/288 [==============================] - 130s 451ms/step - loss: 0.8731 - acc: 0.2006 - val_loss: 0.8618 - val_acc: 0.2037
Epoch 2/50
288/288 [==============================] - 122s 421ms/step - loss: 0.8306 - acc: 0.2192 - val_loss: 0.8331 - val_acc: 0.2311
Epoch 3/50
288/288 [==============================] - 120s 417ms/step - loss: 0.8013 - acc: 0.2406 - val_loss: 0.8108 - val_acc: 0.2470
Epoch 4/50
288/288 [==============================] - 120s 416ms/step - loss: 0.7714 - acc: 0.2612 - val_loss: 0.7843 - val_acc: 0.2617
Epoch 5/50
288/288 [==============================] - 120s 417ms/step - loss: 0.7406 - acc: 0.2763 - val_loss: 0.7604 - val_acc: 0.2771
Epoch 6/50
288/288 [==============================] - 120s 417ms/step - loss: 0.7114 - acc: 0.2917 - val_loss: 0.7382 - val_acc: 0.2936
Epoch 7/50
288/288 [==============================] - 120s 416ms/step - loss: 0.6842 - acc: 0.3094 - val_loss: 0.7180 - val_acc: 0.3109
Epoch 8/50
288/288 [============================

In [135]:
history = 0x7f3d5a662390
print(history)

139901486375824


In [132]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

TypeError: ignored

<Figure size 432x288 with 0 Axes>

# save weights


In [40]:
model.save_weights('nmt_weights.h5')


# load model from weights

In [ ]:
# model.load_weights('nmt_weights.h5')


# inference setup 

In [41]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [43]:
# Image(retina=True, filename='encoder_model.png')

In [45]:
# Image(retina=True, filename='decoder_model.png')


# Decoder sample sequences

In [46]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence



```
# This is formatted as code
```

# evaluation on train dataset

In [47]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [48]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: some parts of this city are very ugly
Actual Marathi Translation:  या शहराचे काही भाग अतिशय कुरूप आहेत 
Predicted Marathi Translation:  या शहराचे काही मोठी खोली आहे 


In [49]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: mary was looking for you at that time
Actual Marathi Translation:  मेरी त्यावेळी तुम्हाला शोधत होती 
Predicted Marathi Translation:  त्यावेळी मेरी स्वतः बघत होतो 


In [50]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: shes used to getting up early
Actual Marathi Translation:  तिला लवकर उठायची सवय आहे 
Predicted Marathi Translation:  ते माझ्या धुवायचे आहेत 


In [51]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he knows everything
Actual Marathi Translation:  त्याला सर्व गोष्टी माहीत आहेत 
Predicted Marathi Translation:  त्यांना सर्वकाही माहीत आहे 


In [52]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: she goes to school
Actual Marathi Translation:  त्या शाळेला जातात 
Predicted Marathi Translation:  त्या शाळेत जातात 


In [53]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: may i ask a question
Actual Marathi Translation:  मी एक प्रश्न विचारू शकते का 
Predicted Marathi Translation:  मी एक प्रश्न विचारू का 


In [54]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: whos that
Actual Marathi Translation:  ते कोण 
Predicted Marathi Translation:  ते कोण आहे 


In [56]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: that wall is cold
Actual Marathi Translation:  ती भिंत थंड आहे 
Predicted Marathi Translation:  ती भिंत थंड आहे 


In [57]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: the noise frightened the baby
Actual Marathi Translation:  त्या आवाजाने बाळ घाबरलं 
Predicted Marathi Translation:  त्या आवाजाने बाळ घाबरलं 


In [58]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: im quite young
Actual Marathi Translation:  मी बर्‍यापैकी तरुण आहे 
Predicted Marathi Translation:  मी अतिशय तरुण आहे 


In [59]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: all of a sudden large drops of rain began falling from the dark sky
Actual Marathi Translation:  अचानक अदीप्त आकाशातून पावसाचे मोठमोठे थेंब पडू लागले 
Predicted Marathi Translation:  अचानक अदीप्त आकाशातून पावसाचे मोठमोठे थेंब पडू ल


In [60]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: do you really want to be a soldier
Actual Marathi Translation:  तुला खरच सैनिक बनायचं आहे का 
Predicted Marathi Translation:  तुला खरच भरपूर मजा झाली आहे का 


In [61]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: theres no hurry
Actual Marathi Translation:  कोणतीही घाई नाही आहे 
Predicted Marathi Translation:  काहीही गरम आहे 


In [62]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: dont put the wet towel in the bag
Actual Marathi Translation:  ओला टॉवेल बॅगेत घालू नकोस 
Predicted Marathi Translation:  ओला टॉवेल पिशवीत घालू नका 


In [63]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i didnt cry that much
Actual Marathi Translation:  मी तितका रडलो नाही 
Predicted Marathi Translation:  मी ते खाल्ला नाही 


In [64]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: why arent you in college
Actual Marathi Translation:  तू कॉलेजमध्ये का नाहीयेस 
Predicted Marathi Translation:  तू कॉलेजमध्ये का नाही आहेस 


In [65]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: wheres my mama
Actual Marathi Translation:  माझी मम्मा कुठे आहे 
Predicted Marathi Translation:  माझी आई कुठे आहे 


In [66]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: do you have many friends
Actual Marathi Translation:  तुझ्याकडे भरपूर मित्रमैत्रिणी आहेत का 
Predicted Marathi Translation:  तुझ्याकडे भरपूर मैत्रिणी आहेत का 


In [67]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom wont ask for help
Actual Marathi Translation:  टॉम मदत मागणार नाही 
Predicted Marathi Translation:  टॉम मदत मागितली नाही 


In [68]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: there are fifteen people here counting the guests
Actual Marathi Translation:  इथे पंधरा लोकं आहेत पाहुण्यांना पकडून 
Predicted Marathi Translation:  इथे पंधरा लोकं आहेत पाहुण्यांना पकडून 


In [69]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i want to be a teacher
Actual Marathi Translation:  मला शिक्षिका बनायचंय 
Predicted Marathi Translation:  मला शिक्षक बनायचं आहे 


In [70]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i didnt know you could speak french so well
Actual Marathi Translation:  तुम्हाला इतक्या बर्‍यापैकी फ्रेंच बोलता येते मला माहीत नव्हतं 
Predicted Marathi Translation:  तू बर्‍यापैकी फ्रेंच बोलता येते तू तुला कधीच सांग


In [71]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: no ones going to buy your story
Actual Marathi Translation:  कोणीही तुझ्या गोष्टीवर विश्वास ठेवणार नाहीये 
Predicted Marathi Translation:  तुमच्या खोलीत नसते तर मी नाही 


In [72]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom has left the country
Actual Marathi Translation:  टॉम देश सोडून गेला आहे 
Predicted Marathi Translation:  टॉम हा जवळच एक द्या 


In [73]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: it was so sweet
Actual Marathi Translation:  किती गोड होतं 
Predicted Marathi Translation:  किती गोड होतं 


In [74]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: im not a saint
Actual Marathi Translation:  मी काय संत नाहीये 
Predicted Marathi Translation:  मी काय संत नाहीये 


In [75]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: you have one chance
Actual Marathi Translation:  तुझ्याकडे एक संधी आहे 
Predicted Marathi Translation:  तुमच्याकडे एक संधी आहे 


In [76]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: whos watching
Actual Marathi Translation:  बघतंय कोण 
Predicted Marathi Translation:  कोण बघतंय 


In [77]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: whats that in your hand
Actual Marathi Translation:  ते तुझ्या हातात काय आहे 
Predicted Marathi Translation:  त्या तुमच्या आजोबांचं काय 


In [78]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: it was the hungry bears that the villagers were afraid of
Actual Marathi Translation:  गावकरांना भीती होती ती भुकेल्या अस्वलांची 
Predicted Marathi Translation:  गावकरांना स्वयंपाकघरात नाही 


In [79]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: why do you want to buy this book
Actual Marathi Translation:  तुम्हाला हे पुस्तक का विकत घ्यायचं आहे 
Predicted Marathi Translation:  तुम्हाला हे पुस्तक हवंय का 


In [80]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: id like to stand up
Actual Marathi Translation:  मी उभं राहू इच्छीतो 
Predicted Marathi Translation:  मला उभं राहू इच्छीते 


In [81]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: whats up dude
Actual Marathi Translation:  काय चाललंय यार 
Predicted Marathi Translation:  काय चाललंय यार 


In [82]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: doesnt tom live in australia
Actual Marathi Translation:  टॉम ऑस्ट्रेलियात नाही राहत का 
Predicted Marathi Translation:  टॉम ऑस्ट्रेलियात राहतो का 


In [83]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: got it
Actual Marathi Translation:  पकडलं 
Predicted Marathi Translation:  समजलं 


In [84]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: im brushing my teeth
Actual Marathi Translation:  मी माझे दात घासतोय 
Predicted Marathi Translation:  मी माझे दात घासतेय 


In [85]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he will come home in a few days
Actual Marathi Translation:  ते काही दिवसांमध्ये घरी येतील 
Predicted Marathi Translation:  ते घरी चार वर्षं लागली का 


In [86]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i want to fix that
Actual Marathi Translation:  आम्हाला ते दुरुस्त करायचं आहे 
Predicted Marathi Translation:  ते मला दुरुस्त करू शकते 


In [87]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: can you repair this
Actual Marathi Translation:  तुम्हाला हे दुरुस्त करता येईल का 
Predicted Marathi Translation:  तू हे दुरुस्त करू शकता का 


# evaluation on test data

In [88]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1


In [89]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: theyll buy anything
Actual Marathi Translation:  ते काहीही विकत घेतील 
Predicted Marathi Translation:  ते विकत घेतली 


In [90]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: my wisdom tooth hurts
Actual Marathi Translation:  माझी अक्कलदाढ दुखतेय 
Predicted Marathi Translation:  माझा दात दुखतोय 


In [91]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: is it hot over there
Actual Marathi Translation:  तिथे गरमी आहे का 
Predicted Marathi Translation:  गरम गरम होतं 


In [92]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: im three years older than you
Actual Marathi Translation:  मी तुझ्यापेक्षा तीन वर्षांनी मोठा आहे 
Predicted Marathi Translation:  मी तुमच्यापेक्षा तीन वर्षांनी मोठा आहे 


In [93]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: do you know who wrote this novel
Actual Marathi Translation:  ही कादंबरी कोणी लिहिली माहीत आहे का 
Predicted Marathi Translation:  हे पुस्तक कोण लिहिलेली कोणती भाषा आहे का 


In [94]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom got big
Actual Marathi Translation:  टॉम मोठा झाला 
Predicted Marathi Translation:  टॉमकडे एक मोठी आहे 


In [95]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: does tom believe in miracles
Actual Marathi Translation:  टॉमचा चमत्कारांवर विश्वास आहे का 
Predicted Marathi Translation:  टॉमचा भुतांवर विश्वास आहे 


In [96]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i cant say
Actual Marathi Translation:  मी सांगू शकत नाही 
Predicted Marathi Translation:  मी नाही नाही करू 


In [97]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i havent decided
Actual Marathi Translation:  मी ठरवलं नाहीये 
Predicted Marathi Translation:  मी ठरवलं नाहीये 


In [98]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i want to see the movie
Actual Marathi Translation:  मला पिक्चर बघायचाय 
Predicted Marathi Translation:  मला पिक्चर बघायला हवा आहे 


In [99]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: it was a terrible movie
Actual Marathi Translation:  बेकार पिक्चर होता 
Predicted Marathi Translation:  तिचं वजन म्हणतात 


In [100]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom is our neighbor
Actual Marathi Translation:  टॉम आपला शेजारी आहे 
Predicted Marathi Translation:  टॉम आमचा शेजारी आहे 


In [101]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom didnt come to the last meeting
Actual Marathi Translation:  टॉम शेवटच्या मीटिंगला आला नाही 
Predicted Marathi Translation:  टॉम गेल्या वर्षी सही केलं 


In [102]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: they began to run when they saw the police
Actual Marathi Translation:  पोलिसांना पाहिल्यावर ते पळू लागले 
Predicted Marathi Translation:  त्यांनी त्या वळून स्वतः स्वतः दिला 


In [103]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: your mother is going to kill me
Actual Marathi Translation:  तुझी आई मला मारून टाकणार आहे 
Predicted Marathi Translation:  तुझी आई यायला लागेल 


In [104]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tell tom what you saw last night
Actual Marathi Translation:  तुम्ही काल रात्री जे पाहिलंत ते टॉमला सांगा 
Predicted Marathi Translation:  काल रात्री तू मला पाहिलं नाही का लक्षात शकतं 


In [105]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he was unwilling to go
Actual Marathi Translation:  ते जायला तयार नव्हते 
Predicted Marathi Translation:  तो जायला तयार झाला 


In [106]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: after a while he came to
Actual Marathi Translation:  थोड्या वेळानंतर त्याला जाग आली 
Predicted Marathi Translation:  तो बाहेर गेले तो एक बाजूला ठेवा 


In [107]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom didnt feel anything
Actual Marathi Translation:  टॉमला काही वाटलं नाही 
Predicted Marathi Translation:  टॉमला काही सांगत नाही 


In [108]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: she gave him something to drink
Actual Marathi Translation:  तिने त्याला काहीतरी प्यायला दिलं 
Predicted Marathi Translation:  तिने काहीही विकत मारणार होतो 


In [109]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: remember it
Actual Marathi Translation:  लक्षात ठेवा 
Predicted Marathi Translation:  समजलं 


In [110]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ill shoot you
Actual Marathi Translation:  मी तुला शूट करतो 
Predicted Marathi Translation:  मी तुम्हाला गोळी मारेन 


In [111]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: how old is your father
Actual Marathi Translation:  तुमच्या वडिलांचं वय काय आहे 
Predicted Marathi Translation:  तुझे वडील किती आहेत 


In [112]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: you look so happy
Actual Marathi Translation:  तू किती खूष दिसतोस 
Predicted Marathi Translation:  तू किती खूष आहात का 


In [113]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: they live in this town
Actual Marathi Translation:  ते या नगरात राहतात 
Predicted Marathi Translation:  त्या शहरात राहतात 


In [114]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: the boy started crying
Actual Marathi Translation:  तो मुलगा रडायला लागला 
Predicted Marathi Translation:  मुलगा रडायला लागला 


In [115]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: an earthquake destroyed the building
Actual Marathi Translation:  एका भुकंपामुळे ती इमारत उध्वस्त झाली 
Predicted Marathi Translation:  एका भुकंपामुळे ती बिल्डिंग चालू होती 


In [116]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: her story is true
Actual Marathi Translation:  त्यांची गोष्ट खरी आहे 
Predicted Marathi Translation:  तिची गोष्ट खरी आहे 


In [117]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: she hugged him
Actual Marathi Translation:  त्यांनी त्याला मिठी मारली 
Predicted Marathi Translation:  तिने त्यांना मिठी मारली 


In [118]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i like to write songs in french
Actual Marathi Translation:  मला फ्रेंचमध्ये गाणी लिहायला आवडतात 
Predicted Marathi Translation:  फ्रेंच फ्रेंचमध्ये वाचायला आवडत 


In [119]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: toms native language is french
Actual Marathi Translation:  टॉमची मातृभाषा फ्रेंच आहे 
Predicted Marathi Translation:  टॉमची फ्रेंच मातृभाषा आहे 


In [120]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: mary is toms motherinlaw
Actual Marathi Translation:  मेरी टॉमची सासू आहे 
Predicted Marathi Translation:  मेरी टॉमचं बाळ आहे 


In [121]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: youre a horrible singer
Actual Marathi Translation:  तुम्ही भयानक गायक आहात 
Predicted Marathi Translation:  तू भयानक गायक आहेस 


In [122]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: prices are about to go up again
Actual Marathi Translation:  भाव परत वाढणार आहेत 
Predicted Marathi Translation:  मिनिट सर्वांना का 


In [123]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he knows how to make a radio
Actual Marathi Translation:  रेडिओ कसा बनवायचा हे त्यांना माहीत आहे 
Predicted Marathi Translation:  त्याला रेडिओ बनवता येतो 


In [124]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom bought a mobile phone
Actual Marathi Translation:  टॉमने एक मोबाईल फोन विकत घेतला 
Predicted Marathi Translation:  टॉमने एक कप एक ग्लास कॅमेरा दे 


In [125]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: everyone else is sleeping
Actual Marathi Translation:  बाकी सर्व झोपले आहेत 
Predicted Marathi Translation:  सगळेच बेडवर जाऊया 


In [126]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he jumped over the hedge
Actual Marathi Translation:  त्याने कुंपणावरून उडी मारली 
Predicted Marathi Translation:  त्यांनी उडी मारली का 


In [127]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: we can still do that
Actual Marathi Translation:  आम्ही अजूनही तसं करू शकतो 
Predicted Marathi Translation:  आम्ही तसं करणार आहे 


In [128]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: turn it off
Actual Marathi Translation:  ते बंद कर 
Predicted Marathi Translation:  बंद करा 


In [129]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i still havent learned to drive a car
Actual Marathi Translation:  मी अजूनही गाडी चालवायला शिकलो नाही आहे 
Predicted Marathi Translation:  मला अजूनही गाडी चालवायला शिकले नाही 
